In [ ]:
from db import executeInput, execute, dropTable
import pandas as pd

# 175. Combine Two Tables

[LeetCode Link](https://leetcode.com/problems/combine-two-tables/) | **Easy** | **Topics:** Joins, Left Join

## Schema

**Table: `Person`**

| Column Name | Type    |
| ----------- | ------- |
| personId    | int     |
| lastName    | varchar |
| firstName   | varchar |

`personId` is the primary key (column with unique values) for this table.
This table contains information about the ID of some persons and their first and last names.

**Table: `Address`**

| Column Name | Type    |
| ----------- | ------- |
| addressId   | int     |
| personId    | int     |
| city        | varchar |
| state       | varchar |

`addressId` is the primary key (column with unique values) for this table.
Each row of this table contains information about the city and state of one person with ID = `PersonId`.

## Problem

Write a solution to report the first name, last name, city, and state of each person in the `Person` table. If the address of a `personId` is not present in the `Address` table, report `null` instead.

Return the result table in any order.


In [87]:
# SETUP
data = [[1, "Wang", "Allen"], [2, "Alice", "Bob"]]
person = pd.DataFrame(data, columns=["personId", "firstName", "lastName"]).astype(
    {"personId": "Int64", "firstName": "object", "lastName": "object"}
)
data = [[1, 2, "New York City", "New York"], [2, 3, "Leetcode", "California"]]
address = pd.DataFrame(data, columns=["addressId", "personId", "city", "state"]).astype(
    {"addressId": "Int64", "personId": "Int64", "city": "object", "state": "object"}
)

executeInput()

[]

In [ ]:
# SQL Solution
query = """
SELECT firstName, lastName, city, state
FROM Person p
LEFT JOIN Address a ON p.personId = a.personID
"""

pd.DataFrame(execute(query))

,0,1,2,3
0,Allen,Wang,None,None
1,Bob,Alice,New York City,New York


In [ ]:
# Cleanup
dropTable("person")
dropTable("address")

In [ ]:
# Pandas solution
import pandas as pd


def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    merged = pd.merge(person, address, on="personId", how="left")
    return merged[["firstName", "lastName", "city", "state"]]

In [125]:
combine_two_tables(person, address)

,firstName,lastName,city,state
0,Wang,Allen,NaN,NaN
1,Alice,Bob,New York City,New York


# 181. Employees Earning More Than Their Managers

[LeetCode Link](https://leetcode.com/problems/employees-earning-more-than-their-managers/) | **Easy** | **Topics:** Database, Join

## Schema

**Table: `Employee`**

| Column Name | Type    |
| ----------- | ------- |
| id          | int     |
| name        | varchar |
| salary      | int     |
| managerId   | int     |

`id` is the primary key (column with unique values) for this table.
Each row of this table indicates the ID of an employee, their name, salary, and the ID of their manager.

## Problem

Write a solution to find the employees who earn more than their managers.

Return the result table in any order.

The result format is in the following example.

## Example 1

**Input:**

`Employee` table:

| id  | name  | salary | managerId |
| --- | ----- | ------ | --------- |
| 1   | Joe   | 70000  | 3         |
| 2   | Henry | 80000  | 4         |
| 3   | Sam   | 60000  | Null      |
| 4   | Max   | 90000  | Null      |

**Output:**

| Employee |
| -------- |
| Joe      |

**Explanation:**
Joe is the only employee who earns more than his manager.


In [135]:
# SETUP
data = [
    [1, "Joe", 70000, 3],
    [2, "Henry", 80000, 4],
    [3, "Sam", 60000, None],
    [4, "Max", 90000, None],
]
employee = pd.DataFrame(data, columns=["id", "name", "salary", "managerId"]).astype(
    {"id": "Int64", "name": "object", "salary": "Int64", "managerId": "Int64"}
)

executeInput()

[]

In [ ]:
# PostgreSQL Soltution
query = """
SELECT e1.name as Employee
FROM Employee e1
JOIN Employee e2 ON e1.managerid = e2.id
WHERE e1.salary > e2.salary
"""
pd.DataFrame(execute(query))

,0
0,Joe


In [ ]:
# cleanup
dropTable("employee")

In [ ]:
# Pandas Solution
import pandas as pd


def find_employees(employee: pd.DataFrame) -> pd.DataFrame:
    merged = pd.merge(
        employee,
        employee,
        left_on="managerId",
        right_on="id",
        suffixes=("", "_manager"),
    )
    res = merged[merged["salary"] > merged["salary_manager"]][["name"]]
    res.columns = ["Employee"]
    return res

In [141]:
find_employees(employee)

,Employee
0,Joe


# 182. Duplicate Emails

[LeetCode Link](https://leetcode.com/problems/duplicate-emails/) | **Easy** | **Topics:** Database

## Schema

**Table: `Person`**

| Column Name | Type    |
| ----------- | ------- |
| id          | int     |
| email       | varchar |

`id` is the primary key (column with unique values) for this table.
Each row of this table contains an email. The emails will not contain uppercase letters.

## Problem

Write a solution to report all the duplicate emails. Note that it's guaranteed that the email field is not NULL.

Return the result table in any order.

The result format is in the following example.

## Example 1

**Input:**

`Person` table:

| id  | email   |
| --- | ------- |
| 1   | a@b.com |
| 2   | c@d.com |
| 3   | a@b.com |

**Output:**

| Email   |
| ------- |
| a@b.com |

**Explanation:**
a@b.com is repeated two times.


In [208]:
# Setup
data = [[1, "a@b.com"], [2, "c@d.com"], [3, "a@b.com"]]
person = pd.DataFrame(data, columns=["id", "email"]).astype(
    {"id": "Int64", "email": "object"}
)

executeInput()

[]

In [ ]:
# PostgreSQL
query = """
select email from person group by email having count(*) > 1
"""

pd.DataFrame(execute(query))

,0
0,a@b.com


In [ ]:
# Cleanup
dropTable("person")

In [ ]:
import pandas as pd


def duplicate_emails(person: pd.DataFrame) -> pd.DataFrame:
    # df = person.groupby('email').id.count().reset_index()
    # df.columns = ['email', 'count']
    # return df[df['count'] > 1][['email']]
    df = person[person.duplicated("email", keep=False)][["email"]]

    return pd.DataFrame({"email": df["email"].unique()})

In [212]:
duplicate_emails(person)

,email
0,a@b.com


# 183. Customers Who Never Order

[LeetCode Link](https://leetcode.com/problems/customers-who-never-order/) | **Easy** | **Topics:** Database, Join

## Schema

**Table: `Customers`**

| Column Name | Type    |
| ----------- | ------- |
| id          | int     |
| name        | varchar |

`id` is the primary key (column with unique values) for this table.
Each row of this table indicates the ID and name of a customer.

**Table: `Orders`**

| Column Name | Type |
| ----------- | ---- |
| id          | int  |
| customerId  | int  |

`id` is the primary key (column with unique values) for this table.
`customerId` is a foreign key (reference column) of the ID from the `Customers` table.
Each row of this table indicates the ID of an order and the ID of the customer who ordered it.

## Problem

Write a solution to find all customers who never order anything.

Return the result table in any order.

The result format is in the following example.

## Example 1

**Input:**

`Customers` table:

| id  | name  |
| --- | ----- |
| 1   | Joe   |
| 2   | Henry |
| 3   | Sam   |
| 4   | Max   |

`Orders` table:

| id  | customerId |
| --- | ---------- |
| 1   | 3          |
| 2   | 1          |

**Output:**

| Customers |
| --------- |
| Henry     |
| Max       |


In [213]:
# Setup
data = [[1, "Joe"], [2, "Henry"], [3, "Sam"], [4, "Max"]]
customers = pd.DataFrame(data, columns=["id", "name"]).astype(
    {"id": "Int64", "name": "object"}
)
data = [[1, 3], [2, 1]]
orders = pd.DataFrame(data, columns=["id", "customerId"]).astype(
    {"id": "Int64", "customerId": "Int64"}
)
executeInput()

[]

In [238]:
# pgSol
query = """
SELECT c.name as Customers
FROM customers c
LEFT JOIN orders o 
    on c.id = o.customerid
WHERE o.customerid is null
"""

In [239]:
pd.DataFrame(execute(query))

,0
0,Henry
1,Max


In [ ]:
dropTable("customers")
dropTable("orders")

Error: table "orders" does not exist



In [ ]:
# pandas sol
import pandas as pd


def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    # df = pd.merge(customers, orders, how='left', left_on='id', right_on='customerId')
    # df = df[df["customerId"].isna()][["name"]]
    # df.columns = ['Customers']
    df = customers[~customers["id"].isin(orders["customerId"])]
    return df[["name"]].rename(columns={"name": "Customers"})

In [ ]:
find_customers(customers, orders)

,Customers
1,Henry
3,Max


# [1757. Recyclable and Low Fat Products](https://leetcode.com/problems/recyclable-and-low-fat-products/description/?envType=study-plan-v2&envId=top-sql-50)

### Description

**Table: `Products`**

| Column Name  | Type |
| :----------- | :--- |
| `product_id` | int  |
| `low_fats`   | enum |
| `recyclable` | enum |

- `product_id` is the primary key (column with unique values) for this table.
- `low_fats` is an ENUM (category) of type `('Y', 'N')` where `'Y'` means this product is low fat and `'N'` means it is not.
- `recyclable` is an ENUM (category) of types `('Y', 'N')` where `'Y'` means this product is recyclable and `'N'` means it is not.

Write a solution to find the ids of products that are both low fat and recyclable.

Return the result table in **any order**.

---

### Example 1

**Input:**

`Products` table:

| product_id | low_fats | recyclable |
| :--------- | :------- | :--------- |
| 0          | Y        | N          |
| 1          | Y        | Y          |
| 2          | N        | Y          |
| 3          | Y        | Y          |
| 4          | N        | N          |

**Output:**

| product_id |
| :--------- |
| 1          |
| 3          |

**Explanation:**  
Only products 1 and 3 are both low fat and recyclable.


In [257]:
data = [
    ["0", "Y", "N"],
    ["1", "Y", "Y"],
    ["2", "N", "Y"],
    ["3", "Y", "Y"],
    ["4", "N", "N"],
]
products = pd.DataFrame(data, columns=["product_id", "low_fats", "recyclable"]).astype(
    {"product_id": "int64", "low_fats": "category", "recyclable": "category"}
)

executeInput()

[]

In [261]:
# postgresql sol
q = """
SELECT product_id
FROM Products
WHERE low_fats='Y' AND recyclable='Y'
"""

In [262]:
pd.DataFrame(execute(q))

,0
0,1
1,3


In [ ]:
# clean up
dropTable("products")

In [ ]:
# pandas
import pandas as pd


def find_products(products: pd.DataFrame) -> pd.DataFrame:
    return products[(products["recyclable"] == "Y") & (products["low_fats"] == "Y")][
        ["product_id"]
    ]

In [273]:
find_products(products)

,product_id
1,1
3,3


# [584. Find Customer Referee](https://leetcode.com/problems/find-customer-referee/?envType=study-plan-v2&envId=top-sql-50)

### Description

**Table: `Customer`**

| Column Name  | Type    |
| :----------- | :------ |
| `id`         | int     |
| `name`       | varchar |
| `referee_id` | int     |

- `id` is the primary key column for this table.
- Each row of this table indicates the `id` of a customer, their `name`, and the `id` of the customer who referred them.

Find the names of the customer that are **not referred by the customer with `id = 2`**.

Return the result table in **any order**.

---

### Example 1

**Input:**

`Customer` table:

| id  | name | referee_id |
| :-- | :--- | :--------- |
| 1   | Will | null       |
| 2   | Jane | null       |
| 3   | Alex | 2          |
| 4   | Bill | null       |
| 5   | Zack | 1          |
| 6   | Mark | 2          |

**Output:**

| name |
| :--- |
| Will |
| Jane |
| Bill |
| Zack |


In [277]:
data = [
    [1, "Will", None],
    [2, "Jane", None],
    [3, "Alex", 2],
    [4, "Bill", None],
    [5, "Zack", 1],
    [6, "Mark", 2],
]
customer = pd.DataFrame(data, columns=["id", "name", "referee_id"]).astype(
    {"id": "Int64", "name": "object", "referee_id": "Int64"}
)
executeInput()

[]

In [285]:
# Postgresql
q = """
SELECT name 
FROM customer
WHERE referee_id IS NULL OR referee_id != 2
"""

In [286]:
pd.DataFrame(execute(q))

,0
0,Will
1,Jane
2,Bill
3,Zack


In [ ]:
# clean up
dropTable("customer")

In [ ]:
# pandas
import pandas as pd


def find_customer_referee(df: pd.DataFrame) -> pd.DataFrame:
    return df[(df["referee_id"] != 2) | (df["referee_id"].isna())][["name"]]

In [294]:
find_customer_referee(customer)

,name
0,Will
1,Jane
3,Bill
4,Zack


# [595. Big Countries](https://leetcode.com/problems/big-countries/?envType=study-plan-v2&envId=top-sql-50)

### Description

**Table: `World`**

| Column Name  | Type    |
| :----------- | :------ |
| `name`       | varchar |
| `continent`  | varchar |
| `area`       | int     |
| `population` | int     |
| `gdp`        | bigint  |

- `name` is the primary key (column with unique values) for this table.
- Each row of this table gives information about the name of a country, the continent to which it belongs, its area, the population, and its GDP value.

A country is **big** if:

- it has an area of at least three million (i.e., `3000000 km2`), or
- it has a population of at least twenty-five million (i.e., `25000000`).

Write a solution to find the name, population, and area of the **big countries**.

Return the result table in **any order**.

---

### Example 1

**Input:**

`World` table:

| name        | continent | area    | population | gdp          |
| :---------- | :-------- | :------ | :--------- | :----------- |
| Afghanistan | Asia      | 652230  | 25500100   | 20343000000  |
| Albania     | Europe    | 28748   | 2831741    | 12960000000  |
| Algeria     | Africa    | 2381741 | 37100000   | 188681000000 |
| Andorra     | Europe    | 468     | 78115      | 3712000000   |
| Angola      | Africa    | 1246700 | 20609294   | 100990000000 |

**Output:**

| name        | population | area    |
| :---------- | :--------- | :------ |
| Afghanistan | 25500100   | 652230  |
| Algeria     | 37100000   | 2381741 |

**Explanation:**

- Afghanistan has a population larger than 25 million.
- Algeria has an area larger than 3 million (Incorrect: The explanation typically notes it meets one of the criteria. Correction based on input: Algeria's population is > 25M, but area < 3M. Wait, checking the input: Algeria area is 2,381,741 which is < 3M. Population is 37,100,000 > 25M. So it qualifies via population.)


In [295]:
data = [
    ["Afghanistan", "Asia", 652230, 25500100, 20343000000],
    ["Albania", "Europe", 28748, 2831741, 12960000000],
    ["Algeria", "Africa", 2381741, 37100000, 188681000000],
    ["Andorra", "Europe", 468, 78115, 3712000000],
    ["Angola", "Africa", 1246700, 20609294, 100990000000],
]
world = pd.DataFrame(
    data, columns=["name", "continent", "area", "population", "gdp"]
).astype(
    {
        "name": "object",
        "continent": "object",
        "area": "Int64",
        "population": "Int64",
        "gdp": "Int64",
    }
)

executeInput()

[]

In [299]:
# postgresql sol

q = """
SELECT
    name,
    population,
    area
FROM
    world
WHERE
    area >= 3000000
    OR population >= 25000000
"""

In [300]:
pd.DataFrame(execute(q))

,0,1,2
0,Afghanistan,25500100,652230
1,Algeria,37100000,2381741


In [ ]:
# cleanup
dropTable("world")

In [ ]:
# pandas
import pandas as pd


def big_countries(world: pd.DataFrame) -> pd.DataFrame:
    return world[(world["area"] >= 3000000) | (world["population"] >= 25000000)][
        ["name", "population", "area"]
    ]

In [306]:
big_countries(world)

,name,population,area
0,Afghanistan,25500100,652230
2,Algeria,37100000,2381741


# [1148. Article Views I](https://leetcode.com/problems/article-views-i/description/?envType=study-plan-v2&envId=top-sql-50)

### Description

**Table: `Views`**

| Column Name  | Type |
| :----------- | :--- |
| `article_id` | int  |
| `author_id`  | int  |
| `viewer_id`  | int  |
| `view_date`  | date |

- There is no primary key (column with unique values) for this table, the table may have duplicate rows.
- Each row of this table indicates that some viewer viewed an article (written by some author) on some date.
- Note that equal `author_id` and `viewer_id` indicate the same person.

Write a solution to find all the authors that viewed at least one of their own articles.

Return the result table sorted by `id` in **ascending order**.

---

### Example 1

**Input:**

`Views` table:

| article_id | author_id | viewer_id | view_date  |
| :--------- | :-------- | :-------- | :--------- |
| 1          | 3         | 5         | 2019-08-01 |
| 1          | 3         | 6         | 2019-08-02 |
| 2          | 7         | 7         | 2019-08-01 |
| 2          | 7         | 6         | 2019-08-02 |
| 4          | 7         | 1         | 2019-07-22 |
| 3          | 4         | 4         | 2019-07-21 |
| 3          | 4         | 4         | 2019-07-21 |

**Output:**

| id  |
| :-- |
| 4   |
| 7   |


In [307]:
data = [
    [1, 3, 5, "2019-08-01"],
    [1, 3, 6, "2019-08-02"],
    [2, 7, 7, "2019-08-01"],
    [2, 7, 6, "2019-08-02"],
    [4, 7, 1, "2019-07-22"],
    [3, 4, 4, "2019-07-21"],
    [3, 4, 4, "2019-07-21"],
]
views = pd.DataFrame(
    data, columns=["article_id", "author_id", "viewer_id", "view_date"]
).astype(
    {
        "article_id": "Int64",
        "author_id": "Int64",
        "viewer_id": "Int64",
        "view_date": "datetime64[ns]",
    }
)
executeInput()

[]

In [312]:
q = """
SELECT DISTINCT
    author_id as id
FROM
    views
WHERE
    author_id = viewer_id;
"""

In [313]:
pd.DataFrame(execute(q))

,0
0,4
1,7


In [ ]:
# cleanup
dropTable("views")

In [365]:
import pandas as pd

def article_views(views: pd.DataFrame) -> pd.DataFrame:
    uniq_authors = views[views["viewer_id"] == views["author_id"]]["author_id"].unique()
    authors = sorted(uniq_authors)
    return pd.DataFrame({"id": authors})

In [366]:
article_views(views)

,id
0,4
1,7
